<a href="https://colab.research.google.com/github/DDricko/Ciencia_de_dados_Ebac/blob/main/Mod5Ex_aulas_01a03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [145]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [146]:
# 2) seu código aqui
missing_per_column = sinasc.isna().sum()
missing_per_column

,0
ORIGEM,0
CODESTAB,115
CODMUNNASC,0
LOCNASC,0
IDADEMAE,0
...,...
munResUf,0
munResLat,1
munResLon,1
munResAlt,1


In [147]:
# 3) seu código aqui
columns_of_interest = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
missing_per_column = sinasc[columns_of_interest].isna().sum()
missing_per_column

,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,317
ESCMAE,312
QTDFILVIVO,1573
GESTACAO,1232
GRAVIDEZ,79
CONSULTAS,0
APGAR5,103


In [148]:
# 4) seu código aqui
sinasc = sinasc.dropna(subset=['APGAR5'])
missing_per_column = sinasc[columns_of_interest].isna().sum()
missing_per_column[columns_of_interest]


,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,315
ESCMAE,310
QTDFILVIVO,1566
GESTACAO,1216
GRAVIDEZ,76
CONSULTAS,0
APGAR5,0


In [149]:
# 5) seu código aqui
print(sinasc['ESTCIVMAE'].unique())

sinasc['ESTCIVMAE'].isna().sum()

estcivmae_vazios = sinasc.loc[sinasc['ESTCIVMAE'].isna() == True]
estcivmae_vazios['ESTCIVMAE']

sinasc['ESTCIVMAE'] = sinasc['ESTCIVMAE'].fillna(9)

estado_civil_dict = {
    1: 'Solteira',
    2: 'Casada',
    3: 'Viúva',
    4: 'Separada/Divorciada',
    5: 'União Estável',
    9: 'Ignorado'
}

# Aplicar a tradução
sinasc['ESTCIVMAE'] = sinasc['ESTCIVMAE'].map(estado_civil_dict)

# Conferir valores
print(sinasc['ESTCIVMAE'].unique())
print(sinasc['ESTCIVMAE'].value_counts())



[ 5.  2.  9.  1.  4. nan  3.]
['União Estável' 'Casada' 'Ignorado' 'Solteira' 'Separada/Divorciada'
 'Viúva']
ESTCIVMAE
Casada                 9906
Solteira               8907
União Estável          7145
Ignorado                459
Separada/Divorciada     451
Viúva                    57
Name: count, dtype: int64


In [150]:
print(sinasc['CONSULTAS'].unique())

sinasc['CONSULTAS'].isna().sum()

consulta_vazia = sinasc.loc[sinasc['CONSULTAS'].isna() == True]
consulta_vazia['CONSULTAS']

sinasc['CONSULTAS'] = sinasc['CONSULTAS'].fillna(9)

consultas_dict = {
    1: 'Nenhuma',
    2: 'de 1 a 3',
    3: 'de 4 a 6',
    4: '7 e mais',
    9: 'Ignorado'
}

sinasc['CONSULTAS'] = sinasc['CONSULTAS'].map(consultas_dict)

print(sinasc['CONSULTAS'].unique())
print(sinasc['CONSULTAS'].value_counts())


[4 3 2 1 9]
['7 e mais' 'de 4 a 6' 'de 1 a 3' 'Nenhuma' 'Ignorado']
CONSULTAS
7 e mais    18587
de 4 a 6     5957
de 1 a 3     1359
Nenhuma       972
Ignorado       50
Name: count, dtype: int64


In [151]:
# 6) Seu código aqui
sinasc['QTDFILVIVO'] = sinasc['QTDFILVIVO'].fillna(0)

In [152]:
# 7) seu código aqui
sinasc['ESCMAE'] = sinasc['ESCMAE'].fillna(sinasc['ESCMAE'].mode()[0])
sinasc['GESTACAO'] = sinasc['GESTACAO'].fillna(sinasc['GESTACAO'].mode()[0])

# Nesse cenario são preenchidos os valores ausentes de ESCMAE e GESTACAO com a moda, pois ambas são variáveis categóricas.
# A média ou mediana não seriam representativas, enquanto a moda mantém a coerência com a categoria mais frequente.


In [153]:
# 8) seu código aqui
sinasc.loc[(sinasc['APGAR5'] >= 8) & (sinasc['APGAR5'] <= 10), 'APGAR5_CAT'] = 'normal'
sinasc.loc[(sinasc['APGAR5'] >= 6) & (sinasc['APGAR5'] <= 7), 'APGAR5_CAT'] = 'asfixia leve'
sinasc.loc[(sinasc['APGAR5'] >= 4) & (sinasc['APGAR5'] <= 5), 'APGAR5_CAT'] = 'asfixia moderada'
sinasc.loc[sinasc['APGAR5'] <= 3, 'APGAR5_CAT'] = 'asfixia severa'
sinasc['APGAR5_CAT'].value_counts()

,count
APGAR5_CAT,
normal,26463
asfixia leve,320
asfixia severa,74
asfixia moderada,68


In [154]:
columns_of_interest = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'APGAR5_CAT']
sinasc[columns_of_interest].head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CAT
0,1,19,União Estável,8 a 11 anos,0.0,37 a 41 semanas,Única,7 e mais,10.0,normal
1,1,29,Casada,8 a 11 anos,1.0,37 a 41 semanas,Única,7 e mais,9.0,normal
2,1,37,Ignorado,8 a 11 anos,2.0,37 a 41 semanas,Única,7 e mais,10.0,normal
3,1,30,União Estável,12 anos ou mais,0.0,37 a 41 semanas,Única,de 4 a 6,10.0,normal
4,1,30,Casada,8 a 11 anos,1.0,37 a 41 semanas,Única,7 e mais,10.0,normal


In [142]:
# 9) seu código aqui
sinasc.columns = sinasc.columns.str.lower().str.replace(' ', '_')
sinasc.columns


Index(['origem', 'codestab', 'codmunnasc', 'locnasc', 'idademae', 'estcivmae',
       'escmae', 'codocupmae', 'qtdfilvivo', 'qtdfilmort', 'codmunres',
       'gestacao', 'gravidez', 'parto', 'consultas', 'dtnasc', 'horanasc',
       'sexo', 'apgar1', 'apgar5', 'racacor', 'peso', 'idanomal', 'dtcadastro',
       'codanomal', 'numerolote', 'versaosist', 'dtrecebim', 'difdata',
       'dtrecoriga', 'naturalmae', 'codmunnatu', 'codufnatu', 'escmae2010',
       'seriescmae', 'dtnascmae', 'racacormae', 'qtdgestant', 'qtdpartnor',
       'qtdpartces', 'idadepai', 'dtultmenst', 'semagestac', 'tpmetestim',
       'consprenat', 'mesprenat', 'tpapresent', 'sttrabpart', 'stcesparto',
       'tpnascassi', 'tpfuncresp', 'tpdocresp', 'dtdeclarac', 'escmaeagr1',
       'stdnepidem', 'stdnnova', 'codpaisres', 'tprobson', 'paridade',
       'kotelchuck', 'contador', 'munresstatus', 'munrestipo', 'munresnome',
       'munresuf', 'munreslat', 'munreslon', 'munresalt', 'munresarea',
       'apgar5_cat'],
 